In [1]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import time
import math
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector
from plyer import notification
from gtts import gTTS
from IPython.display import Audio, display

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Initialize HandDetector
detector = HandDetector(detectionCon=0.8, maxHands=2)

# Function to calculate Euclidean distance
def euclidean_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to check for gestures
def check_gesture(hand, lip_center):
    fingers = detector.fingersUp(hand)
    if fingers == [1, 0, 0, 0, 0]:  # Thumb up, other fingers down
        thumb_tip = hand['lmList'][4]  # Thumb tip landmark
        distance = euclidean_distance(thumb_tip[:2], lip_center)
        if distance <= 50:  # Distance threshold in pixels (assuming 1 pixel ≈ 0.1 cm)
            return "Patient Wants Water"
    elif fingers == [0, 1, 1, 0, 0]:
        return "Patient Wants to Defecate"
    elif fingers == [0, 0, 0, 0, 1]:
        return "Patient Wants to Void"
    elif fingers == [0, 0, 0, 0, 0]:
        return "Patient Needs Urgent Help"
    elif fingers == [1, 1, 1, 1, 1]:  # All fingers up
        return "Patient Needs Help"
    return None

# Function to notify the patient

def notify_input(text):
    # Generate the voice message
    message=f"{text.lower()} is intimated to the nurse"
    tts = gTTS(text=message, lang='en')
    tts.save('input_taken.mp3')

    # Display and play the voice message
    display(Audio('input_taken.mp3', autoplay=True))
    

    return text

# Initialize webcam
cap = cv2.VideoCapture(0)

gesture_start_time = None
gesture_duration_threshold = 5  # 5 seconds
notification_interval = 5  # Notify every 5 seconds
notification_last_sent = time.time()  # Initialize with current time

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    lip_center = None
    neck_center = None
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            lips = [face_landmarks.landmark[13], face_landmarks.landmark[14]]  # Upper and lower lips
            lip_center = [(lips[0].x + lips[1].x) / 2 * frame.shape[1], (lips[0].y + lips[1].y) / 2 * frame.shape[0]]
            neck = [face_landmarks.landmark[152], face_landmarks.landmark[152]]  # Using a single landmark for neck
            neck_center = [neck[0].x * frame.shape[1], neck[0].y * frame.shape[0]]

    # Detect hands in the frame
    hands, img = detector.findHands(frame, draw=True)

    if len(hands) == 2 and neck_center is not None:
        hand1_center = hands[0]['center']
        hand2_center = hands[1]['center']
        distance1_to_neck = euclidean_distance(hand1_center, neck_center)
        distance2_to_neck = euclidean_distance(hand2_center, neck_center)
        if distance1_to_neck <= 100 and distance2_to_neck <= 100:
            gesture = "Patient is Choking"
            cv2.putText(img, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if gesture_start_time is None:
                gesture_start_time = time.time()  # Start the timer
            elif time.time() - gesture_start_time >= gesture_duration_threshold:
                # Check if the notification interval has passed
                if time.time() - notification_last_sent >= notification_interval:
                    notify_input(gesture)
                    # Throw a notification
                    notification.notify(
                        title='Notification',
                        message=gesture,
                        timeout=10
                    )
                    # Update the last notification time
                    notification_last_sent = time.time()
        else:
            gesture_start_time = None  # Reset timer if gesture is not detected

    elif hands and lip_center is not None:
        for hand in hands:
            hand_center = hand['center']
            distance_to_nose = euclidean_distance(hand_center, lip_center)
            if distance_to_nose <= 350:  # 100mm in pixels (assuming 1 pixel ≈ 0.1 mm)
                gesture = check_gesture(hand, lip_center)
                if gesture:
                    cv2.putText(img, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    if gesture_start_time is None:
                        gesture_start_time = time.time()  # Start the timer
                    elif time.time() - gesture_start_time >= gesture_duration_threshold:
                        # Check if the notification interval has passed
                        if time.time() - notification_last_sent >= notification_interval:
                            notify_input(gesture)
                            # Throw a notification
                            notification.notify(
                                title='Notification',
                                message=gesture,
                                timeout=10
                            )
                            # Update the last notification time
                            notification_last_sent = time.time()
                else:
                    gesture_start_time = None  # Reset timer if gesture is not detected
            else:
                gesture_start_time = None  # Reset timer if hand is not within 100mm distance from the nose tip

    # Check for key presses
    key = cv2.waitKey(10) & 0xFF
    if key == ord('q'):
        break

    # Display the frame
    cv2.imshow('Hand Gesture Recognition', img)

cap.release()
cv2.destroyAllWindows()


In [3]:
cap.release()
cv2.destroyAllWindows()